In [35]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

In [36]:
### Print functions
def solve_print():
    m.reset()
    m.optimize()
    if m.Status == GRB.OPTIMAL:
        print("Total Profit = {0:.2f} ".format(m.ObjVal))
        f.write("Total Profit, {}".format(m.ObjVal))
        csv_out = np.zeros([m.numvars,5])
        row_label = []
        for var in m.getVars(): row_label.append(var.varname)
        column_label = ['X', 'Obj', 'rc', 'Objlow','ObjUp']

        tp=0
        for var in m.getVars():
            csv_out[tp,0] = var.x
            csv_out[tp,1] = var.obj
            csv_out[tp,2] = var.rc
            csv_out[tp,3] = var.saobjlow
            csv_out[tp,4] = var.saobjup
            tp += 1
        df = pd.DataFrame(csv_out, columns=column_label, index=row_label)
        f.write('\n')
        df.to_csv(f,mode='a')
        m.printAttr(['X', 'Obj', 'rc', 'saobjlow', 'saobjup'])
        
        csv_out = np.zeros([m.numconstrs,5])
        row_label = []
        for constr in m.getConstrs(): row_label.append(constr.constrname)
        column_label = ['RHS', 'Slack','pi', 'RHSlow','RHSup']

        tp = 0
        for const in m.getConstrs():
            csv_out[tp,0] = const.RHS
            csv_out[tp,1] = const.slack
            csv_out[tp,2] = const.pi
            csv_out[tp,3] = const.sarhslow
            csv_out[tp,4] = const.sarhsup
            tp += 1   
        df = pd.DataFrame(csv_out, columns=column_label, index=row_label)
        f.write("\n")
        df.to_csv(f,mode='a')
        f.write("\n")
        m.printAttr(['RHS', 'Sense', 'Slack', 'pi', 'sarhslow', 'sarhsup' ])

    else: print('Could not solve model')

In [37]:
### Sensitivity functions
def set_obj (i, j):
    m.getVars()[i].obj = j

    
def set_rhs(i,j):
    m.getConstrs()[i].rhs=j    
    
def new_obj_calc(i,j):
    save = m.getVars()[i].obj
    if j>0: new = objup[i] + epsilon
    else: new = objlow[i] - epsilon
    print("\n\n New Obj ",m.getVars()[i].VarName," = ",new)
    set_obj(i,new)
    solve_print()
    set_obj(i,save)
    m.update()

    
def new_rhs_calc(i,j):
    save = m.getConstrs()[i].rhs
    if j>0: new = rhsup[i] + epsilon
    else: new = rhslow[i] - epsilon 
    print("\n\n New RHS ",m.getConstrs()[i].ConstrName," = ",new)
    set_rhs(i,new)
    solve_print()
    set_rhs(i,save)
    m.update()

In [38]:
########################################
### Main logic
########################################

epsilon = 0.0001
inf = 1e30
f = open("temp.csv","w", newline='')

m = gp.Model("Crispy Critters")
m.ModelSense = GRB.MAXIMIZE

In [39]:
###
### Variables
###
num_vars = 12  
var_names = ['A','B','EA','EB','A1','A2','B1','B2','EA1','EA2','EB1','EB2']

Cost = np.array([-11,-17.5,-15,-21.5,82,87.5,82,87.5,75.5,81,75.5,81])

x = m.addMVar(num_vars, obj=Cost, lb=0, name=var_names)
m.update()

In [40]:
###
### Constraints
###

# A + B <= 1000, EA + EB <= 400, A1 + B1 + A2 + B2 <= 800, EA1 + EA2 + EB1 + EB2 <= 300
A = np.array([
    [1,1,0,0,0,0,0,0,0,0,0,0],
    [0,0,1,1,0,0,0,0,0,0,0,0],
    [0,0,0,0,1,1,1,1,0,0,0,0],
    [0,0,0,0,0,0,0,0,1,1,1,1]
     ])
rhs =np.array([1000, 400, 800, 300])  
m.addMConstr(A , x, '<', rhs, name='Harvest & Chemical')

# A1 + A2 + EA1 + EA2 <= 0.4(A + EA)
# B1 + B2 +  EB1 + EB2 <= 0.7(B + EB)
A = np.array([
    [0.4,0,0.4,0,-1,-1,0,0,-1,-1,0,0],
    [0,0.7,0,0.7,0,0,-1,-1,0,0,-1,-1]
     ])
rhs =np.array([0,0])  
m.addMConstr(A , x, '>', rhs, name='Mix')

# good
# 0.5(A1 + EA1) + 0.4(A2 + EA2) + 0.6(B1 + EB1) + 0.3(B2 + EB2) 
# >=  0.4 (A1 + B1 + A2 + B2 + EA1 + EA2 + EB1 + EB2)
A = np.array([
    [0,0,0,0,0.1,0,0.2,-0.1,0.1,0,0.2,-0.1]
     ])
rhs =np.array([0])  
m.addMConstr(A , x, '>', rhs, name='Good')

# bad
#0.05(A1 + EA1) + 0.06(A2 + EA2) + 0.1(B1 + EB1) + 0.06(B2 + EB2) 
#<=  0.07(A1 + B1 +  A2 + B2 + EA1 + EA2 + EB1 + EB2)
A = np.array([
    [0,0,0,0,-0.02,-0.01,0.03,-0.01,-0.02,-0.01,0.03,-0.01]
     ])
rhs =np.array([0])  
m.addMConstr(A , x, '<', rhs, name='Bad')

m.update()

In [41]:
### 
### Script for results
###

solve_print()

# save the original problem details before sensitivity
objup = m.getAttr('SaObjUp')
objlow = m.getAttr('SaObjlow')
rhsup = m.getAttr('SaRhsUp')
rhslow = m.getAttr('SaRhsLow')


f.close()

Discarded solution information
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8 rows, 12 columns and 38 nonzeros
Model fingerprint: 0xc976cc50
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [1e+01, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve time: 0.00s
Presolved: 8 rows, 12 columns, 38 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.5200000e+32   1.600000e+31   6.520000e+02      0s
       9    5.2933617e+04   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.293361702e+04
Total Profit = 52933.62 

    Variable            X          Obj           rc     saobjlow      saobjup 
-----------------------------------------------------------------------------
           A      208.511          -11            0      